<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Process-Data" data-toc-modified-id="Process-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Process Data</a></span></li><li><span><a href="#Pre-Process-Data-For-Deep-Learning" data-toc-modified-id="Pre-Process-Data-For-Deep-Learning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pre-Process Data For Deep Learning</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Look-at-one-example-of-processed-issue-bodies" data-toc-modified-id="Look-at-one-example-of-processed-issue-bodies-2.0.0.1"><span class="toc-item-num">2.0.0.1&nbsp;&nbsp;</span>Look at one example of processed issue bodies</a></span></li><li><span><a href="#Look-at-one-example-of-processed-issue-titles" data-toc-modified-id="Look-at-one-example-of-processed-issue-titles-2.0.0.2"><span class="toc-item-num">2.0.0.2&nbsp;&nbsp;</span>Look at one example of processed issue titles</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Define-Model-Architecture" data-toc-modified-id="Define-Model-Architecture-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define Model Architecture</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Load-the-data-from-disk-into-variables" data-toc-modified-id="Load-the-data-from-disk-into-variables-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Load the data from disk into variables</a></span></li><li><span><a href="#Define-Model-Architecture" data-toc-modified-id="Define-Model-Architecture-3.0.2"><span class="toc-item-num">3.0.2&nbsp;&nbsp;</span>Define Model Architecture</a></span></li></ul></li></ul></li><li><span><a href="#Train-Model" data-toc-modified-id="Train-Model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Train Model</a></span></li><li><span><a href="#See-Results-On-Holdout-Set" data-toc-modified-id="See-Results-On-Holdout-Set-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>See Results On Holdout Set</a></span></li><li><span><a href="#Feature-Extraction-Demo" data-toc-modified-id="Feature-Extraction-Demo-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Feature Extraction Demo</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Example-1:-Issues-Installing-Python-Packages" data-toc-modified-id="Example-1:-Issues-Installing-Python-Packages-6.0.1"><span class="toc-item-num">6.0.1&nbsp;&nbsp;</span>Example 1: Issues Installing Python Packages</a></span></li><li><span><a href="#Example-2:--Issues-asking-for-feature-improvements" data-toc-modified-id="Example-2:--Issues-asking-for-feature-improvements-6.0.2"><span class="toc-item-num">6.0.2&nbsp;&nbsp;</span>Example 2:  Issues asking for feature improvements</a></span></li></ul></li></ul></li></ul></div>

In [1]:
%env GCE_PROJECT_NAME 'My First Project'
%env TPU_ZONE us-central1-f
%env TPU_NAME digitransolutions1
%env TPU_IP_RANGE 10.240.1.0/29
%env GCS_DATA_PATH gs://digitran-data-bucket/mnist/
%env GCS_CKPT_PATH gs://digitran-checkpoint-bucket/mnist/
    
# Automatically get bucket name from GCS paths
import os
os.environ['GCS_DATA_BUCKET'] = os.environ['GCS_DATA_PATH'][5:].split('/')[0]
os.environ['GCS_CKPT_BUCKET'] = os.environ['GCS_CKPT_PATH'][5:].split('/')[0]

env: GCE_PROJECT_NAME='My First Project'
env: TPU_ZONE=us-central1-f
env: TPU_NAME=digitransolutions1
env: TPU_IP_RANGE=10.240.1.0/29
env: GCS_DATA_PATH=gs://digitran-data-bucket/mnist/
env: GCS_CKPT_PATH=gs://digitran-checkpoint-bucket/mnist/


In [2]:
!gcloud config set compute/zone $TPU_ZONE
!gcloud alpha compute tpus create $TPU_NAME --range=$TPU_IP_RANGE --accelerator-type=tpu-v2 --version=nightly --zone=$TPU_ZONE

Updated property [compute/zone].
ERROR: (gcloud.alpha.compute.tpus.create) ALREADY_EXISTS: Resource 'projects/modern-impulse-280709/locations/us-central1-f/nodes/digitransolutions1' already exists
- '@type': type.googleapis.com/google.rpc.ResourceInfo
  resourceName: projects/modern-impulse-280709/locations/us-central1-f/nodes/digitransolutions1


In [3]:
!gsutil mb -c regional -l us-central1 gs://$GCS_DATA_BUCKET
!gsutil mb -c regional -l us-central1 gs://$GCS_CKPT_BUCKET

!gsutil iam ch serviceAccount:`gcloud alpha compute tpus describe $TPU_NAME | grep serviceAccount | cut -d' ' -f2`:admin gs://$GCS_DATA_BUCKET gs://$GCS_CKPT_BUCKET && echo 'Successfully set permissions!'

Creating gs://digitran-data-bucket/...
ServiceException: 409 Bucket digitran-data-bucket already exists.
Creating gs://digitran-checkpoint-bucket/...
ServiceException: 409 Bucket digitran-checkpoint-bucket already exists.
No changes made to gs://digitran-data-bucket/
No changes made to gs://digitran-checkpoint-bucket/
Successfully set permissions!


In [4]:
DATA_PATH = 'gs://digitran-data-bucket/'

In [5]:
import tensorflow as tf
# Detect hardware
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError: # If TPU not found
  tpu = None
# Select appropriate distribution strategy
if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
else:
  strategy = tf.distribute.get_strategy() # Default strategy that works on CPU and single GPU
  print('Running on CPU instead')
print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Initializing the TPU system: digitransolutions1


INFO:tensorflow:Initializing the TPU system: digitransolutions1


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  ['10.240.1.2:8470']
Number of accelerators:  8


In [14]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1EzcbYa4aRvLcTCn6xhlP1iG8fL8baNsb' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1EzcbYa4aRvLcTCn6xhlP1iG8fL8baNsb" -O github_issues.csv && rm -rf /tmp/cookies.txt

--2020-06-24 22:35:47--  https://docs.google.com/uc?export=download&confirm=dDnn&id=1EzcbYa4aRvLcTCn6xhlP1iG8fL8baNsb
Resolving docs.google.com (docs.google.com)... 172.217.212.102, 172.217.212.100, 172.217.212.101, ...
Connecting to docs.google.com (docs.google.com)|172.217.212.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-8k-docs.googleusercontent.com/docs/securesc/gbb1teob90hbs7efjfek4aaer8c6b9cu/jsgf1dl3ailrd7q9l9aps94opao62ua1/1593038100000/09218841784108975906/08623641984017747873Z/1EzcbYa4aRvLcTCn6xhlP1iG8fL8baNsb?e=download [following]
--2020-06-24 22:35:47--  https://doc-0k-8k-docs.googleusercontent.com/docs/securesc/gbb1teob90hbs7efjfek4aaer8c6b9cu/jsgf1dl3ailrd7q9l9aps94opao62ua1/1593038100000/09218841784108975906/08623641984017747873Z/1EzcbYa4aRvLcTCn6xhlP1iG8fL8baNsb?e=download
Resolving doc-0k-8k-docs.googleusercontent.com (doc-0k-8k-docs.googleusercontent.com)... 173.194.197.132, 2607:f8b0:4001:c1b::84
Conne

In [6]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'seq2seq-model/'
import sys
sys.path.append(base_dir)

ModuleNotFoundError: No module named 'google.colab'

In [7]:
!pip install astor
!pip install wget
!pip install nmslib
!pip install pathos
!pip install pandas
!pip install more_itertools
!pip install spacy
!python3 -m spacy download en
!pip install ktext
!pip install annoy
!pip install fastai
!pip install torch
!pip install torchtext
!pip install torchvision
import tqdm
from tqdm import tqdm_notebook
!pip install scikit-learn==0.20.2

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/digitransolutions1/.local/lib/python3.7/site-packages/en_core_web_sm -->
/home/digitransolutions1/.local/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')
Defaulting to user installation because normal site-pac

In [6]:
import pandas as pd
import logging
import glob
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 500)
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

In [7]:
from pathlib import Path
# INPUT_PATH = Path(base_dir)
base_dir = 'data/'
INPUT_PATH = Path(base_dir)
INPUT_FILE = Path(INPUT_PATH/'github_issues.csv')
print("INPUT_FILE = ", INPUT_FILE)

INPUT_FILE =  data/github_issues.csv


# Process Data

Look at filesystem to see files extracted from BigQuery (or Kaggle: https://www.kaggle.com/davidshinn/github-issues/)

In [ ]:
!ls -lah | grep INPUT_FILE

Split data into train and test set and preview data

In [7]:
with strategy.scope(): 
    #read in data sample 2M rows (for speed of tutorial)
    traindf, testdf = train_test_split(pd.read_csv(INPUT_FILE).sample(n=1200000), test_size=.10)


    #print out stats about shape of data
    print(f'Train: {traindf.shape[0]:,} rows {traindf.shape[1]:,} columns')
    print(f'Test: {testdf.shape[0]:,} rows {testdf.shape[1]:,} columns')

    # preview data
    traindf.head(3)

Train: 1,080,000 rows 3 columns
Test: 120,000 rows 3 columns


**Convert to lists in preparation for modeling**

In [8]:
with strategy.scope(): 
    train_body_raw = traindf.body.tolist()
    train_title_raw = traindf.issue_title.tolist()
    #preview output of first element
    train_body_raw[0]

# Pre-Process Data For Deep Learning

See [this repo](https://github.com/hamelsmu/ktext) for documentation on the ktext package

In [9]:
%reload_ext autoreload
%autoreload 2
from ktext.preprocess import processor

Using TensorFlow backend.


In [10]:
%%time
# Clean, tokenize, and apply padding / truncating such that each document length = 70
#  also, retain only the top 8,000 words in the vocabulary and set the remaining words
#  to 1 which will become common index for rare words 
with strategy.scope(): 
    body_pp = processor(keep_n=8000, padding_maxlen=70)
    train_body_vecs = body_pp.fit_transform(train_body_raw)

CPU times: user 2min 3s, sys: 6.78 s, total: 2min 10s
Wall time: 4min 56s


#### Look at one example of processed issue bodies

In [11]:
print('\noriginal string:\n', train_body_raw[0], '\n')
print('after pre-processing:\n', train_body_vecs[0], '\n')


original string:
 it would be cool if the sliders volume and track timing were working. and for example, a click on the cover opened the main page 

after pre-processing:
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0   10   43   15 1877   21    3    1 1384    9 1074 4001
  412  214    9   14   96    5  264   16    3 2240  995    3  182   93] 



In [12]:
# Instantiate a text processor for the titles, with some different parameters
#  append_indicators = True appends the tokens '_start_' and '_end_' to each
#                      document
#  padding = 'post' means that zero padding is appended to the end of the 
#             of the document (as opposed to the default which is 'pre')
title_pp = processor(append_indicators=True, keep_n=4500, 
                     padding_maxlen=12, padding ='post')

# process the title data
train_title_vecs = title_pp.fit_transform(train_title_raw)

#### Look at one example of processed issue titles

In [13]:
print('\noriginal string:\n', train_title_raw[0])
print('after pre-processing:\n', train_title_vecs[0])


original string:
 sliders feature request
after pre-processing:
 [ 2  1 55 43  3  0  0  0  0  0  0  0]


In [14]:
OUTPUT_PATH = Path(DATA_PATH)
OUTPUT_PATH.mkdir(exist_ok=True)

FileNotFoundError: [Errno 2] No such file or directory: 'data/data/seq2seq'

Serialize all of this to disk for later use

In [32]:
import dill as dpickle
import numpy as np

# Save the preprocessor
with open('seq2seq/body_pp.dpkl', 'wb') as f:
    dpickle.dump(body_pp, f)

with open('seq2seq/title_pp.dpkl', 'wb') as f:
    dpickle.dump(title_pp, f)

# Save the processed data
np.save('seq2seq/train_title_vecs.npy', train_title_vecs)
np.save('seq2seq/train_body_vecs.npy', train_body_vecs)

In [34]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.4 MB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 661 kB 9.8 MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434673 sha256=727c0f7c410fab7c530f8555255b56a9b8dd767aa08b6577dcc42b8c8398ce59
  Stored in directory: /home/digitransolutions1/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk


# Define Model Architecture

### Load the data from disk into variables

In [8]:
from seq2seq_utils import load_decoder_inputs, load_encoder_inputs, load_text_processor

Using TensorFlow backend.


In [9]:
encoder_input_data, doc_length = load_encoder_inputs('seq2seq/train_body_vecs.npy')
decoder_input_data, decoder_target_data = load_decoder_inputs('seq2seq/train_title_vecs.npy')

Shape of encoder input: (1080000, 70)
Shape of decoder input: (1080000, 11)
Shape of decoder target: (1080000, 11)


In [10]:
num_encoder_tokens, body_pp = load_text_processor('seq2seq/body_pp.dpkl')
num_decoder_tokens, title_pp = load_text_processor('seq2seq/title_pp.dpkl')

Size of vocabulary for seq2seq/body_pp.dpkl: 8,002
Size of vocabulary for seq2seq/title_pp.dpkl: 4,502


### Define Model Architecture

In [38]:
%matplotlib inline
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Embedding, Bidirectional, BatchNormalization
from tensorflow.keras import optimizers
from seq2seq_utils import viz_model_architecture
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
with strategy.scope(): 
  #arbitrarly set latent dimension for embedding and hidden units
  latent_dim = 300

  ##### Define Model Architecture ######

  ########################
  #### Encoder Model ####
  encoder_inputs = Input(shape=(doc_length,), name='Encoder-Input')

  # Word embeding for encoder (ex: Issue Body)
  x = Embedding(num_encoder_tokens, latent_dim, name='Body-Word-Embedding', mask_zero=False)(encoder_inputs)
  x = BatchNormalization(name='Encoder-Batchnorm-1')(x)

  # Intermediate GRU layer (optional)
  #x = GRU(latent_dim, name='Encoder-Intermediate-GRU', return_sequences=True)(x)
  #x = BatchNormalization(name='Encoder-Batchnorm-2')(x)

  # We do not need the `encoder_output` just the hidden state.
  _, state_h = GRU(latent_dim, return_state=True, name='Encoder-Last-GRU')(x)

  # Encapsulate the encoder as a separate entity so we can just 
  #  encode without decoding if we want to.
  encoder_model = Model(inputs=encoder_inputs, outputs=state_h, name='Encoder-Model')

  seq2seq_encoder_out = encoder_model(encoder_inputs)

  ########################
  #### Decoder Model ####
  decoder_inputs = Input(shape=(None,), name='Decoder-Input')  # for teacher forcing

  # Word Embedding For Decoder (ex: Issue Titles)
  dec_emb = Embedding(num_decoder_tokens, latent_dim, name='Decoder-Word-Embedding', mask_zero=False)(decoder_inputs)
  dec_bn = BatchNormalization(name='Decoder-Batchnorm-1')(dec_emb)

  # Set up the decoder, using `decoder_state_input` as initial state.
  decoder_gru = GRU(latent_dim, return_state=True, return_sequences=True, name='Decoder-GRU')
  decoder_gru_output, _ = decoder_gru(dec_bn, initial_state=seq2seq_encoder_out)
  x = BatchNormalization(name='Decoder-Batchnorm-2')(decoder_gru_output)

  # Dense layer for prediction
  decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='Final-Output-Dense')
  decoder_outputs = decoder_dense(x)

  ########################
  #### Seq2Seq Model ####

  #seq2seq_decoder_out = decoder_model([decoder_inputs, seq2seq_encoder_out])
  
  
  

In [39]:
with strategy.scope(): 
  seq2seq_Model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
  seq2seq_Model.compile(optimizer=optimizers.Nadam(lr=0.001), loss='sparse_categorical_crossentropy')
  seq2seq_Model.summary() 
  script_name_base = 'tutorial_seq2seq'
  csv_logger = CSVLogger('{:}.log'.format(script_name_base))
  model_checkpoint = ModelCheckpoint('{:}.epoch{{epoch:02d}}-val{{val_loss:.5f}}.hdf5'.format(script_name_base),
                                   save_best_only=True)

  batch_size = 1200
  epochs = 7
  history = seq2seq_Model.fit([encoder_input_data, decoder_input_data], np.expand_dims(decoder_target_data, -1),
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.12, callbacks=[csv_logger, model_checkpoint])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder-Input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
Decoder-Word-Embedding (Embeddi (None, None, 300)    1350600     Decoder-Input[0][0]              
__________________________________________________________________________________________________
Encoder-Input (InputLayer)      [(None, 70)]         0                                            
__________________________________________________________________________________________________
Decoder-Batchnorm-1 (BatchNorma (None, None, 300)    1200        Decoder-Word-Embedding[0][0]     
______________________________________________________________________________________________

In [43]:
seq2seq_Model.compile(optimizer=optimizers.Nadam(lr=0.00001), loss='sparse_categorical_crossentropy')
batch_size = 1200
epochs = 1
history = seq2seq_Model.fit([encoder_input_data, decoder_input_data], np.expand_dims(decoder_target_data, -1),
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.12, callbacks=[csv_logger, model_checkpoint])

Train on 950400 samples, validate on 129600 samples
950400/950400 [==============================] - 68s 72us/sample - loss: 1.8455 - val_loss: 2.4658


** Examine Model Architecture Summary **

# Train Model

In [44]:
#save model
seq2seq_Model.save('seq2seq/seq2seq_model_tutorial.h5')

In [11]:
from tensorflow.keras.models import load_model
seq2seq_Model=load_model('seq2seq/seq2seq_model_tutorial.h5')

# See Results On Holdout Set

In [49]:
from seq2seq_utils import Seq2Seq_Inference
seq2seq_inf = Seq2Seq_Inference(encoder_preprocessor=body_pp,
                                 decoder_preprocessor=title_pp,
                                 seq2seq_model=seq2seq_Model)

In [50]:
# this method displays the predictions on random rows of the holdout set
seq2seq_inf.demo_model_predictions(n=50, issue_df=testdf)



============== Example # 65817 =================

"https://github.com/Cisco-Talos/snort-faq/issues/13"
Issue Body:
 i'm doing research trying to figure out how to add users using the -u along with snort but i can't find any documentation on how or where to create the rule to add a new userid 

Original Title:
 how to add a user to snort

****** Machine Generated Title (Prediction) ******:
 how to add a new user


============== Example # 33225 =================

"https://github.com/autolab/Autolab/issues/916"
Issue Body:
 we should recommend text in annotations, since graders likely type the same responses over and over 

Original Title:
 hotkeys like gradescope

****** Machine Generated Title (Prediction) ******:
 recommend annotations for a text box


============== Example # 4011 =================

"https://github.com/Virtual-Labs/molecular-florescence-spectroscopy-responsive-lab-iiith/issues/85"
Issue Body:
 defect description: the quiz page of effects of excimer and exciplex for

# Feature Extraction Demo

In [12]:
with strategy.scope(): 
    # Read All 5M data points
    all_data_df = pd.read_csv(INPUT_FILE).sample(n=1200000)
    # Extract the bodies from this dataframe
    all_data_bodies = all_data_df['body'].tolist()

In [ ]:
with strategy.scope(): 
    # transform all of the data using the ktext processor
    all_data_vectorized = body_pp.transform_parallel(all_data_bodies)

In [ ]:
# save transformed data
with open('all_data_vectorized.dpkl', 'wb') as f:
    dpickle.dump(all_data_vectorized, f)

In [ ]:
%reload_ext autoreload
%autoreload 2
from seq2seq_utils import Seq2Seq_Inference
seq2seq_inf_rec = Seq2Seq_Inference(encoder_preprocessor=body_pp,
                                    decoder_preprocessor=title_pp,
                                    seq2seq_model=seq2seq_Model)
recsys_annoyobj = seq2seq_inf_rec.prepare_recommender(all_data_vectorized, all_data_df)

### Example 1: Issues Installing Python Packages

In [ ]:
seq2seq_inf_rec.demo_model_predictions(n=1, issue_df=testdf, threshold=1)



============== Example # 13563 =================

"https://github.com/bnosac/pattern.nlp/issues/5"
Issue Body:
 thanks for your package, i can't wait to use it. unfortunately i have issues with the installation. prerequisite is 'first install python version 2.5+ not version 3 '. so this package cant be used with version 3.6 64bit that i have installed? i nevertheless tried to install it using pip, conda is not supported? but got an error: 'syntaxerror: missing parentheses in call to 'print''. besides when i try to run the library in r version 3.3.3. 64 bit i got errors with can_find_python_cmd required_modules = pattern.db : 'error in find_python_cmd......' pattern seems to be written in python but must be used in r, why cant it be used in python? i found another python pattern application that apparently does the same in python: https://pypi.python.org/pypi/pattern how is this related? 

Original Title:
 error installation python

****** Machine Generated Title (Prediction) ******:


,issue_url,issue_title,body,dist
286906,"""https://github.com/scikit-hep/root_numpy/issues/337""",root 6.10/02 and root_numpy compatibility,i am trying to pip install root_pandas and one of the dependency is root_numpy however some weird reasons i am unable to install it even though i can import root in python. i am working on python3.6 as i am more comfortable with it. is root_numpy is not yet compatible with the latest root?,0.694671
314005,"""https://github.com/andim/noisyopt/issues/4""",joss review: installing dependencies via pip,"hi, i'm trying to install noisyopt in a clean conda environment running python 3.5. running pip install noisyopt does not install the dependencies numpy, scipy . i see that you do include a requires keyword argument in your setup.py file, does this need to be install_requires ? as in https://packaging.python.org/requirements/ . also, not necessary if you don't want to, but i think it would be good to include a list of dependences somewhere in the readme.",0.698265
48120,"""https://github.com/turi-code/SFrame/issues/389""",python 3.6 compatible,"hi: i tried to install sframe using pip and conda but i can not find anything that will work with python 3.6? has sframe been updated to work with python 3.6 yet? thanks, drew",0.718715


### Example 2:  Issues asking for feature improvements

In [ ]:
seq2seq_inf_rec.demo_model_predictions(n=1, issue_df=testdf, threshold=1)



============== Example # 157322 =================

"https://github.com/Chingu-cohorts/devgaido/issues/89"
Issue Body:
 right now, your profile link is https://devgaido.com/profile. this is fine, but it would be really cool if there was a way to share your profile with other people. on my portfolio, i have social media buttons to freecodecamp, github, ect. without a custom link, i cannot show-off what i have done on devgaido to future employers. 

Original Title:
 feature request: sharable profile.

****** Machine Generated Title (Prediction) ******:
 add a link to your profile

**** Similar Issues (using encoder embedding) ****:



,issue_url,issue_title,body,dist
250423,"""https://github.com/ParabolInc/action/issues/1379""",integrations list view discoverability,"issue - enhancement i was initially confused by the link to my account copy; seeing github in the integrations list made me think it had already been set up . i realize now that i had to allow parabol to post as me. i think that link to my account could use a tooltip explaining what link means, and why you'd want to do so. <img width= 728 alt= screen shot 2017-09-29 at 10 52 05 am src= https://user-images.githubusercontent.com/2146312/31024786-2fd39c46-a50e-11e7-9f2a-6d4a5ed2baeb.png >",0.748828
222304,"""https://github.com/viosey/hexo-theme-material/issues/166""",allow us to use sns-share for github,"i'd love to be able to add a link at the bottom of the page for my github account. however, the sns-share option doesn't currently seem to be able to do this.",0.774398
153327,"""https://github.com/tobykurien/GoogleApps/issues/31""",drive provide download ability,sometimes people share files via g drive. provided a link this app can show some info about the files but doesn't show the download button. i hope that it can be fixed and users would be able to download files with this app.,0.778953


In [ ]:
# incase you need to reset the rec system
# seq2seq_inf_rec.set_recsys_annoyobj(recsys_annoyobj)
# seq2seq_inf_rec.set_recsys_data(all_data_df)

# save object
recsys_annoyobj.save('recsys_annoyobj.pkl')

True